In [1]:
import steamdata
import preprocess

In [3]:
app_ids = [2586870, 1497440, 1036240, 1902690, 1607250]
df = steamdata.get_data(app_ids)
df

Loading idprocessed_on_20231001.txt
Skipping previously found appID = 2586870
Skipping previously found appID = 1497440
Skipping previously found appID = 1036240
Skipping previously found appID = 1902690
Skipping previously found appID = 1607250
Game records written: 0


,app_id,recommendationid,review,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count
0,2586870,147379813,My friend who I play CS:GO with said if I post...,True,4,0,0.196229293942451477,0
1,2586870,147376402,I really loved playing through the prologue! P...,True,1,0,0.523809552192687988,0
2,2586870,147372568,Soul's Spectrum: Awakening is a game that real...,True,0,0,0,0
3,2586870,147369693,The traps and puzzles in the game are so varie...,True,0,0,0,0
4,2586870,147364446,Soul's Spectrum: Awakening is a free prequel t...,True,0,0,0,0
...,...,...,...,...,...,...,...,...
2239,1607250,147386006,"No gay sex, zero stars",False,0,0,0,0
2240,1607250,147385883,Super fun but gacha bad.,True,0,0,0,0
2241,1607250,147385803,add nude patch and i will play it.,False,4,3,0.406651943922042847,0
2242,1607250,147385492,"OK game, fun for a bit but then just kinda sta...",False,1,0,0.502463042736053467,0


In [4]:
df = preprocess.get_preprocessed_data(df)
df

,app_id,recommendationid,review,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,review_preprocessed
0,2586870,147379813,My friend who I play CS:GO with said if I post...,True,4,0,0.196229293942451477,0,my friend who i play cs:go with said if i post...
1,2586870,147376402,I really loved playing through the prologue! P...,True,1,0,0.523809552192687988,0,i really loved playing through the prologue! p...
2,2586870,147372568,Soul's Spectrum: Awakening is a game that real...,True,0,0,0,0,soul's spectrum: awakening is a game that real...
3,2586870,147369693,The traps and puzzles in the game are so varie...,True,0,0,0,0,the traps and puzzles in the game are so varie...
4,2586870,147364446,Soul's Spectrum: Awakening is a free prequel t...,True,0,0,0,0,soul's spectrum: awakening is a free prequel t...
...,...,...,...,...,...,...,...,...,...
2239,1607250,147386006,"No gay sex, zero stars",False,0,0,0,0,"no gay sex, zero stars"
2240,1607250,147385883,Super fun but gacha bad.,True,0,0,0,0,super fun but gacha bad.
2241,1607250,147385803,add nude patch and i will play it.,False,4,3,0.406651943922042847,0,add nude patch and i will play it.
2242,1607250,147385492,"OK game, fun for a bit but then just kinda sta...",False,1,0,0.502463042736053467,0,"ok game, fun for a bit but then just kinda sta..."
